In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Activation
from keras.optimizers import SGD
from keras.layers import Dense
from keras.utils import np_utils
from keras.layers import Input, Dense
from keras.models import Sequential,Model,load_model

Using TensorFlow backend.


# **Entity Linking w Babelfy**

In [0]:
!pip install babelpy

  Created wheel for babelpy: filename=BabelPy-1.0.1-cp36-none-any.whl size=9723 sha256=444738923927fc0ccd348caaaea235163cecb87ed2b190868f477a871f0ff26d
  Stored in directory: /root/.cache/pip/wheels/c9/53/db/2b25b9bdc97574bd1a076b31d0a8ba8f8ce0969e247debcc5b
Successfully built babelpy


In [0]:
from babelpy import babelfy

In [0]:
#from babelfy import BabelfyClient


# Instantiate BabelFy client

API_KEY  = 'ddacf5fe-702e-4b24-b6b8-95a4806d8acc'

params = dict()
params['lang'] = 'EN'
babel_client = babelfy.BabelfyClient(API_KEY, params)

In [0]:
text = "earth sky dancing intelligent many levels co writers costa st john ramirez deliver blend indigenous soul r b afro latin rhythms smooth fun deep justo almario horns handle oh yea"

In [0]:
def babelfyReview(x):

  # Babelfy sentence
  babel_client.babelfy(x)

  # Get entity data
  #print(babel_client.entities)
  try:
    #entitiesFlag = True
    #print('here')
    entitiesDictList = babel_client.entities
    #print(babel_client.entities)
  except AttributeError:
    ("AttributeError for review: "+x)
    return(x)

  entitiesToAdd = []

  #if(entitiesFlag):
  #print(entitiesDictList)
  for entDict in entitiesDictList:
    entitiesToAdd.append(entDict['text'])

  entities = ' '.join(entitiesToAdd)
  x = x +" "+entities

  return(x)



In [0]:
text = babelfyReview(text)

[{'start': 0, 'end': 4, 'text': 'earth', 'isEntity': True, 'tokenFragment': {'start': 0, 'end': 0}, 'charFragment': {'start': 0, 'end': 4}, 'babelSynsetID': 'bn:00029424n', 'DBpediaURL': 'http://dbpedia.org/resource/Earth', 'BabelNetURL': 'http://babelnet.org/rdf/s00029424n', 'score': 0.9195402298850575, 'coherenceScore': 0.4, 'globalScore': 0.09148084619782733, 'source': 'BABELFY'}, {'start': 6, 'end': 8, 'text': 'sky', 'isEntity': True, 'tokenFragment': {'start': 1, 'end': 1}, 'charFragment': {'start': 6, 'end': 8}, 'babelSynsetID': 'bn:00072016n', 'DBpediaURL': 'http://dbpedia.org/resource/Sky', 'BabelNetURL': 'http://babelnet.org/rdf/s00072016n', 'score': 0.0, 'coherenceScore': 0.0, 'globalScore': 0.0, 'source': 'MCS'}, {'start': 10, 'end': 16, 'text': 'dancing', 'isEntity': True, 'tokenFragment': {'start': 2, 'end': 2}, 'charFragment': {'start': 10, 'end': 16}, 'babelSynsetID': 'bn:00086271v', 'DBpediaURL': '', 'BabelNetURL': 'http://babelnet.org/rdf/s00086271v', 'score': 1.0, 'co

In [0]:
text

'earth sky dancing intelligent many levels co writers costa st john ramirez deliver blend indigenous soul r b afro latin rhythms smooth fun deep justo almario horns handle oh yea earth sky dancing intelligent levels writers st john john deliver blend indigenous soul afro latin latin rhythms rhythms smooth fun deep justo almario horns handle'

In [0]:
df = pd.read_pickle("reviews_cleaned.pkl")
#df1000 = df.sample(n = 1000)
df1000_babelfied = pd.read_pickle("df1000_babelfied.pkl")

In [0]:
df.head()

,amazonIDs,reviews,genres,clean_reviews
0,1458389375,"Earth and Sky Dancing Music, intelligent on so...","Jazz,Pop",earth sky dancing intelligent many levels co w...
1,1591791065,I bought this based on the snippets that they ...,"New Age,Dance Pop,World Music,Pop,Classical",bought based snippets give amazon actual c muc...
2,1906063443,"Contrary to the previous review, this IS a com...","Europe,Christian,Eastern Europe,Pop,Gypsy,Worl...",contrary previous review compilation single ar...
3,1929243766,My family and I all first experienced John on ...,"Comedy & Spoken Word,Pop",family first experienced john comedy central t...
4,1930864159,Awesome. This is so good when needing reminded...,"Pop & Contemporary,Christian,Gospel,Pop",awesome good needing reminded quiet hear god w...


In [0]:
df1000_babelfied.head()

,amazonIDs,reviews,genres,clean_reviews
2398,B000001320,Primitive Streak is the last studio recording ...,"Americana,Country,Pop,Adult Alternative,Roots ...",primitive streak last studio recording made su...
1485,B000000NMZ,I'm not here to review the music on this album...,"Progressive,House,Pop,New Age,Dance & Electron...",im review rather quality specific release manu...
1220,B000000H97,"for weeks, track three was the anthem of my sl...","Electronica,Dance & Electronic,Techno,Pop,Ambient",weeks three anthem sleep sweetly sad melody om...
2382,B0000012FW,Psychedelicatessen is Threshold's second offer...,"Progressive Rock,Metal,Progressive,Pop,Rock",psychedelicatessen thresholds second offering ...
1995,B000000XHF,"The modus operandi of Ellington Is Forever, Vo...","Jazz,Pop,Soul-Jazz & Boogaloo",modus operandi ellington forever vol essential...


In [0]:
remove_babelfied_entries = df1000_babelfied.reviews.tolist()
df = df[~df['reviews'].isin(remove_babelfied_entries)]


In [0]:
# get another sample of 1000 entries to use with Babelfy API, totaling 2000 "bablefied" entries
df2000 = df.sample(n = 1000)

In [0]:
df2000["clean_reviews"] = df2000["clean_reviews"].apply(lambda x: babelfyReview(x))

KeyboardInterrupt: ignored

In [0]:
df2000 = df1000_babelfied.append(df2000)

df2000.to_pickle("df2000_babelfied.pkl")

In [0]:
df2000

,amazonIDs,reviews,genres,clean_reviews
2398,B000001320,Primitive Streak is the last studio recording ...,"Americana,Country,Pop,Adult Alternative,Roots ...",primitive streak last studio recording made su...
1485,B000000NMZ,I'm not here to review the music on this album...,"Progressive,House,Pop,New Age,Dance & Electron...",im review rather quality specific release manu...
1220,B000000H97,"for weeks, track three was the anthem of my sl...","Electronica,Dance & Electronic,Techno,Pop,Ambient",weeks three anthem sleep sweetly sad melody om...
2382,B0000012FW,Psychedelicatessen is Threshold's second offer...,"Progressive Rock,Metal,Progressive,Pop,Rock",psychedelicatessen thresholds second offering ...
1995,B000000XHF,"The modus operandi of Ellington Is Forever, Vo...","Jazz,Pop,Soul-Jazz & Boogaloo",modus operandi ellington forever vol essential...
...,...,...,...,...
2181,B000000Z2E,It's on this album that Sonny really begins a ...,"Jazz Fusion,R&B,Jazz,Pop,Modern Postbebop,Funk...",sonny really begins journey toward sound refer...
439,B000000545,This is just as solid as his first and then th...,"Gangsta & Hardcore,Pop Rap,Rap & Hip-Hop,West ...",solid first b legit showing world clicks sucse...
858,B000000DX3,this is a most excellent insight into the earl...,"Reggae,World Music,Pop",excellent insight earlier years reggae version...
854,B000000DWJ,Bunny Wailer's heavy commentary on Apartheid m...,"Reggae,World Music,Pop",bunny wailers heavy commentary apartheid makes...


In [0]:
df2000 = pd.read_pickle("df2000_babelfied.pkl")

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer

def changeToTuple(x):
    stringListGenres = x.split(",")
    stringListGenres = tuple(stringListGenres)
    return(stringListGenres)

y = df2000["genres"]
y = y.apply(lambda x: changeToTuple(x))

targetList = y.tolist()

mlb = MultiLabelBinarizer()

targetLabelsBin = mlb.fit_transform(targetList)

In [0]:
targetLabelsBin

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [0]:
!pip install scipy

In [4]:
import sys
import numpy
#numpy.set_printoptions(threshold=sys.maxsize)
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csc_matrix

tfidf_vectorizer = TfidfVectorizer(max_features=10000)
X_train, X_test, y_train, y_test = train_test_split(df2000['clean_reviews'], targetLabelsBin, test_size=0.2, random_state=9)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = tfidf_vectorizer.transform(X_test).toarray()
print(len(y_train[0]))
print(X_train_tfidf.shape)
print(X_train_tfidf[0])
y_train.shape
print(X_test_tfidf.shape)

327
(1600, 10000)
[0. 0. 0. ... 0. 0. 0.]
(400, 10000)


In [0]:
whos

Variable              Type                   Data/Info
------------------------------------------------------
API_KEY               str                    ddacf5fe-702e-4b24-b6b8-95a4806d8acc
Activation            type                   <class 'keras.layers.core.Activation'>
Adam                  type                   <class 'keras.optimizers.Adam'>
Dense                 type                   <class 'keras.layers.core.Dense'>
Flatten               type                   <class 'keras.layers.core.Flatten'>
Input                 function               <function Input at 0x7fa3356fa268>
LabelEncoder          type                   <class 'sklearn.preproces<...>ing._label.LabelEncoder'>
Model                 type                   <class 'keras.engine.training.Model'>
MultiLabelBinarizer   type                   <class 'sklearn.preproces<...>bel.MultiLabelBinarizer'>
SGD                   type                   <class 'keras.optimizers.SGD'>
Sequential            type                   <

# **Building Simple Neural Net**

In [5]:
# creating a simple feed forward network

from keras.layers import Input, Dense, Flatten, Dropout

input_tfidf = Input(shape=(10000,))
dense1 = Dense(2048, activation = 'relu')(input_tfidf)
dense2 = Dense(2048, activation = 'relu')(dense1)
output = Dense(327, activation = 'sigmoid')(dense2)

model = Model(input = input_tfidf, output = output)

#similar sequential model

modelSeq  = Sequential()
modelSeq.add(Dense(2048, input_shape = (10000,), activation = 'relu'))
#modelSeq.add(Dropout(0.25))
modelSeq.add(Dense(2048, activation = 'relu'))
#modelSeq.add(Dropout(0.10))
modelSeq.add(Dense(327, activation = 'sigmoid'))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.


In [0]:
# auc function from stackoverflow user tuomastik
# currently not using this ---> error being caused with this function?
import tensorflow as tf
from keras import backend as K

def auc(y_true, y_pred):
    auc = tf.metrics.AUC(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [0]:
from keras.optimizers import SGD, Adam

adam = Adam(lr=0.0001)
model.compile(optimizer = 'adam',metrics = ['accuracy', tf.keras.metrics.AUC(multi_label = True)], loss = 'binary_crossentropy')
modelSeq.compile(optimizer = 'adam',metrics = ['accuracy', tf.keras.metrics.AUC(multi_label = True)], loss = 'binary_crossentropy')

In [0]:
modelSeq.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 2048)              20482048  
_________________________________________________________________
dense_17 (Dense)             (None, 2048)              4196352   
_________________________________________________________________
dense_18 (Dense)             (None, 327)               670023    
Total params: 25,348,423
Trainable params: 25,348,423
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(X_train_tfidf, y_train, epochs = 20, batch_size = 32, verbose = 1, validation_split=0.2)

Train on 1280 samples, validate on 320 samples
Epoch 1/20
1280/1280 [==============================] - 20s 16ms/step - loss: 0.1994 - accuracy: 0.9682 - auc: 0.3951 - val_loss: 0.0815 - val_accuracy: 0.9805 - val_auc: 0.4820
Epoch 2/20
1280/1280 [==============================] - 18s 14ms/step - loss: 0.0790 - accuracy: 0.9815 - auc: 0.5079 - val_loss: 0.0785 - val_accuracy: 0.9805 - val_auc: 0.5227
Epoch 3/20
1280/1280 [==============================] - 15s 12ms/step - loss: 0.0719 - accuracy: 0.9816 - auc: 0.5309 - val_loss: 0.0774 - val_accuracy: 0.9807 - val_auc: 0.5378
Epoch 4/20
1280/1280 [==============================] - 19s 15ms/step - loss: 0.0681 - accuracy: 0.9822 - auc: 0.5470 - val_loss: 0.0741 - val_accuracy: 0.9813 - val_auc: 0.5552
Epoch 5/20
1280/1280 [==============================] - 16s 13ms/step - loss: 0.0596 - accuracy: 0.9836 - auc: 0.5661 - val_loss: 0.0679 - val_accuracy: 0.9820 - val_auc: 0.5781
Epoch 6/20
1280/1280 [==============================] - 17s 13m

# **Testing on specific examples, getting scores, investigating output**

In [0]:
ynew = modelSeq.predict_classes(X_test_tfidf)

In [0]:
preds = model.predict(X_test_tfidf)
preds[preds>=0.5] = 1
preds[preds<0.5] = 0

In [0]:
preds[25]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0.

In [0]:
result = np.where(preds[25] == 1.)
result

(array([221, 249]),)

In [0]:
result2 = np.where(y_test[25] == 1)
result2

(array([ 12,  90, 110, 221, 293]),)

In [10]:
model.evaluate(X_test_tfidf, y_test)

400/400 [==============================] - 1s 3ms/step


[0.13617886781692504, 0.9843807220458984, 0.831349790096283]

In [0]:
modelSeq.metrics_names

['loss', 'accuracy']

In [13]:
from sklearn.metrics import label_ranking_average_precision_score

label_ranking_average_precision_score(y_test, preds)

0.32190892723590553

In [12]:
from sklearn.metrics import average_precision_score

preds = model.predict(X_test_tfidf)

preds[preds>=0.5] = 1
preds[preds<0.5] = 0

average_precision_score(y_test, preds)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_ranking.py:677: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


nan

In [0]:
# use this to check for class imbalances with "support" numbers
from sklearn.metrics import classification_report

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00        15
           3       0.00      0.00      0.00        20
           4       0.00      0.00      0.00         7
           5       0.00      0.00      0.00         5
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         9
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00         6
          11       0.89      0.51      0.65        83
          12       0.00      0.00      0.00         7
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00        21
          15       0.00      0.00      0.00         3
          16       0.00      0.00      0.00         6
          17       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Label ranking average precision (LRAP) averages over the samples the answer to the following question: for each ground truth label, what fraction of higher-ranked labels were true labels? This performance measure will be higher if you are able to give better rank to the labels associated with each sample. The obtained score is always strictly greater than 0, and the best value is 1. If there is exactly one relevant label per sample, label ranking average precision is equivalent to the mean reciprocal rank.